# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'regression_energy_cleanLevel4_trainX_regression_GRU_rg_'
}

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_GRU_rg_'
}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_'
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params3

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_param'] = model_meta['scaler_param']
params['transform_param'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# 3. Data Preparation
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'classification_actionPattern_cleanLevel0_testX', 'feature_list': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']}


window_size: 128 nan_limit_num: 64
(177280, 9) (1385, 128, 9)
(1385, 1) (1385, 1)


In [7]:
# set scaler by data y flag
if params['data_y_flag']:
    scaler = scaler_y
else:
    scaler = scaler_X

In [8]:
# model test
result = ML_api.ML_test(params, test_X_array, test_y_array, scaler)


Start testing data

** Performance of test dataset ==> PROB = [[9.99998927e-01 5.15355687e-08 2.95214221e-07 3.03272031e-07
  2.97835385e-07 2.96314568e-07]
 [9.99999523e-01 1.19106724e-08 1.45155113e-07 1.47288631e-07
  1.40893377e-07 1.41825055e-07]
 [7.93707477e-06 9.99982238e-01 2.45680576e-06 2.36880351e-06
  2.32258799e-06 2.58611658e-06]
 ...
 [9.99998450e-01 6.90028372e-08 3.23694451e-07 3.27218771e-07
  3.13480911e-07 3.18369246e-07]
 [9.99999523e-01 1.11920722e-08 1.49558019e-07 1.53559895e-07
  1.45239582e-07 1.45953834e-07]
 [3.23663861e-03 9.96522903e-01 6.04581874e-05 5.94143639e-05
  5.80887536e-05 6.25389948e-05]], ACC = 0.9861918604651163
** Dimension of result for test dataset = (1376,)
scale
=========================data
      value  prediction
0       0.0         0.0
1       0.0         0.0
2       5.0         5.0
3       0.0         0.0
4       0.0         0.0
...     ...         ...
1371    0.0         0.0
1372    0.0         0.0
1373    0.0         0.0
1374    0

In [9]:
result['result_metrics']

{'0': {'precision': 0.9836206896551725,
  'recall': 1.0,
  'f1-score': 0.9917427205562799,
  'support': 1141},
 '1': {'precision': 1.0,
  'recall': 0.9191489361702128,
  'f1-score': 0.9578713968957872,
  'support': 235},
 'accuracy': 0.9861918604651163,
 'macro avg': {'precision': 0.9918103448275862,
  'recall': 0.9595744680851064,
  'f1-score': 0.9748070587260336,
  'support': 1376},
 'weighted avg': {'precision': 0.9864180282678429,
  'recall': 0.9861918604651163,
  'f1-score': 0.9859580104834486,
  'support': 1376}}

In [10]:
model_meta['model_info']

{'model_purpose': 'classification',
 'model_method': 'GRU_cf',
 'model_name': 'classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'input_size': 9,
  'seq_len': 128,
  'output_dim': 1,
  'num_classes': 6,
  'num_layers': 2,
  'hidden_size': 64,
  'dropout': 0.1,
  'bidirectional': True,
  'rnn_type': 'gru'},
 'seq_len': 128,
 'input_size': 9,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/GRU_cf/classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_/classification_actionPattern_cleanLevel0_trainX/model.pkl'}}}